In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q kaggle opendatasets pillow matplotlib seaborn scikit-learn
!pip install -q pytorch-grad-cam timm albumentations
!pip install -q wandb tensorboard  # Experiment tracking
!pip install -q grad-cam opencv-python
!pip install -q efficientnet-pytorch


In [ ]:
import os
import sys
import json
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import shutil
from tqdm.auto import tqdm
from datetime import datetime
from collections import defaultdict
import cv2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
from torch.cuda.amp import autocast, GradScaler

# Sklearn
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve,
                             f1_score, accuracy_score)
from sklearn.model_selection import StratifiedKFold

# Albumentations for advanced augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Model interpretability
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image

print(f" PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")
    print(f" GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

#

In [ ]:
 ========================================== 
# SECTION 3: CONFIGURATION & HYPERPARAMETERS
# ==========================================
class Config:
    # Paths
    BASE_DIR = '/content/xray_dataset'
    CHECKPOINT_DIR = '/content/checkpoints'
    LOG_DIR = '/content/logs'
    
    # Model
    MODEL_NAME = 'efficientnet_b3'  # Options: efficientnet_b0-b7, resnet50, densenet121
    NUM_CLASSES = 4
    IMG_SIZE = 256  # Increased for better feature extraction
    PRETRAINED = True
    
    # Training
    BATCH_SIZE = 24
    NUM_EPOCHS = 25
    LEARNING_RATE = 3e-4
    WEIGHT_DECAY = 1e-4
    WARMUP_EPOCHS = 3
    
    # Augmentation
    USE_ADVANCED_AUG = True
    MIXUP_ALPHA = 0.2
    CUTMIX_ALPHA = 1.0
    
    # Optimization
    USE_MIXED_PRECISION = True
    GRADIENT_CLIP = 1.0
    EARLY_STOPPING_PATIENCE = 7
    
    # Validation
    K_FOLDS = 5
    TEST_TIME_AUGMENTATION = True
    TTA_ITERATIONS = 5
    
    # Classes
    CLASSES = ['NORMAL', 'PNEUMONIA', 'COVID', 'TB']
    
    # Random seed
    SEED = 42

# Create directories
os.makedirs(Config.CHECKPOINT_DIR, exist_ok=True)
os.makedirs(Config.LOG_DIR, exist_ok=True)

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(Config.SEED)
print(" Configuration loaded & seeds set!")

# ========================================== 
# SECTION 4: EXPERIMENT LOGGER
# ==========================================
class ExperimentLogger:
    def __init__(self, log_dir):
        self.log_dir = log_dir
        self.metrics = defaultdict(list)
        self.start_time = time.time()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.log_file = os.path.join(log_dir, f'experiment_{timestamp}.json')
        
    def log(self, phase, epoch, metrics_dict):
        """Log metrics for an epoch"""
        metrics_dict['epoch'] = epoch
        metrics_dict['phase'] = phase
        metrics_dict['timestamp'] = time.time() - self.start_time
        
        for key, value in metrics_dict.items():
            self.metrics[f"{phase}_{key}"].append(value)
        
        # Save to file
        with open(self.log_file, 'w') as f:
            json.dump(dict(self.metrics), f, indent=2)
    
    def get_best_metric(self, metric_name):
        """Get best value of a metric"""
        values = self.metrics.get(metric_name, [])
        return max(values) if values else 0.0

logger = ExperimentLogger(Config.LOG_DIR)
print(f" Logger initialized: {logger.log_file}")


In [ ]:
# SECTION 5: DATASET WITH ADVANCED FEATURES
# ==========================================
# Download datasets (same as before, condensed)
import opendatasets as od
print(" Downloading datasets...")
datasets = [
    "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia",
    "https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database",
    "https://www.kaggle.com/datasets/raddar/tuberculosis-chest-xrays-shenzhen"
]
for dataset in datasets:
    try:
        od.download(dataset)
    except:
        print(f"⚠️ Failed to download {dataset}")

# Organize dataset
def organize_dataset():
    base_dir = Config.BASE_DIR
    os.makedirs(base_dir, exist_ok=True)
    
    for split in ['train', 'val', 'test']:
        for category in Config.CLASSES:
            os.makedirs(f'{base_dir}/{split}/{category}', exist_ok=True)
    
    def move_images(source_paths, label, ratios=(0.7, 0.15, 0.15)):
        all_images = []
        for path in source_paths:
            if os.path.exists(path):
                imgs = [os.path.join(path, f) for f in os.listdir(path) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                all_images.extend(imgs)
        
        np.random.shuffle(all_images)
        n1 = int(len(all_images) * ratios[0])
        n2 = int(len(all_images) * ratios[1])
        
        splits = {
            'train': all_images[:n1],
            'val': all_images[n1:n1+n2],
            'test': all_images[n1+n2:]
        }
        
        for split, imgs in splits.items():
            dest = f'{base_dir}/{split}/{label}'
            for i, img in enumerate(tqdm(imgs, desc=f'{label} {split}')):
                try:
                    shutil.copy(img, f'{dest}/{label}_{i}.jpg')
                except:
                    continue
        
        return len(splits['train']), len(splits['val']), len(splits['test'])
    
    # Organize all classes
    stats = {}
    stats['NORMAL'] = move_images([
        '/content/chest-xray-pneumonia/chest_xray/train/NORMAL',
        '/content/chest-xray-pneumonia/chest_xray/test/NORMAL'
    ], 'NORMAL')
    
    stats['PNEUMONIA'] = move_images([
        '/content/chest-xray-pneumonia/chest_xray/train/PNEUMONIA',
        '/content/chest-xray-pneumonia/chest_xray/test/PNEUMONIA'
    ], 'PNEUMONIA')
    
    stats['COVID'] = move_images([
        '/content/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images'
    ], 'COVID')
    
    stats['TB'] = move_images([
        '/content/tuberculosis-chest-xrays-shenzhen/images'
    ], 'TB')
    
    print("\n Dataset Statistics:")
    print(pd.DataFrame(stats, index=['Train', 'Val', 'Test']))
    return stats

dataset_stats = organize_dataset()


In [ ]:
 ========================================== 
# SECTION 6: ADVANCED AUGMENTATION PIPELINE
# ==========================================
def get_train_transforms(img_size=256):
    """Advanced augmentation with Albumentations"""
    return A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=15, p=0.5),
        A.OneOf([
            A.ElasticTransform(alpha=120, sigma=120 * 0.05, p=1),
            A.GridDistortion(p=1),
            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
        ], p=0.3),
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=1),
            A.GaussianBlur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1),
        ], p=0.2),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def get_val_transforms(img_size=256):
    """Validation transforms"""
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


In [ ]:
# ========================================== 
# SECTION 7: CUSTOM DATASET WITH CACHING
# ==========================================
class AdvancedXRayDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, cache=True):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.classes = Config.CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.cache = cache
        self.cache_dict = {}
        
        # Load all paths
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, split, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        path = os.path.join(class_dir, img_name)
                        self.samples.append((path, self.class_to_idx[class_name]))
        # ========================================== 
# SECTION 7: CUSTOM DATASET WITH CACHING
# ==========================================
class AdvancedXRayDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, cache=True):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.classes = Config.CLASSES
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.cache = cache
        self.cache_dict = {}
        
        # Load all paths
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, split, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        path = os.path.join(class_dir, img_name)
                        self.samples.append((path, self.class_to_idx[class_name]))
        
        print(f"Loaded {len(self.samples)} images for {split} set")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Check cache
        if self.cache and img_path in self.cache_dict:
            image = self.cache_dict[img_path]
        else:
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.cache:
                self.cache_dict[img_path] = image
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, label
        print(f"Loaded {len(self.samples)} images for {split} set")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Check cache
        if self.cache and img_path in self.cache_dict:
            image = self.cache_dict[img_path]
        else:
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.cache:
                self.cache_dict[img_path] = image
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, label

In [ ]:

# ========================================== 
# SECTION 8: MULTI-MODEL ARCHITECTURE
# ==========================================
class AttentionBlock(nn.Module):
    """Channel Attention Module"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class AdvancedXRayModel(nn.Module):
    def __init__(self, model_name='efficientnet_b3', num_classes=4, pretrained=True):
        super().__init__()
        
        # Load backbone
        if 'efficientnet' in model_name:
            if model_name == 'efficientnet_b0':
                self.backbone = models.efficientnet_b0(pretrained=pretrained)
                num_features = 1280
            elif model_name == 'efficientnet_b3':
                self.backbone = models.efficientnet_b3(pretrained=pretrained)
                num_features = 1536
            else:
                raise ValueError(f"Model {model_name} not supported")
            
            # Add attention before classifier
            self.backbone.classifier = nn.Identity()
            self.attention = AttentionBlock(num_features)
            
        elif model_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=pretrained)
            num_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.attention = AttentionBlock(num_features)
        
        # Custom classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        features = self.backbone(x)
        
        # If features are 4D, apply attention
        if len(features.shape) == 4:
            features = self.attention(features)
            features = F.adaptive_avg_pool2d(features, 1).flatten(1)
        
        return self.classifier(features)

In [ ]:

# ========================================== 
# SECTION 9: ADVANCED TRAINING UTILITIES
# ==========================================
def mixup_data(x, y, alpha=0.2):
    """Mixup augmentation"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Mixup loss"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance"""
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.alpha is not None:
            focal_loss = self.alpha[targets] * focal_loss
            
        return focal_loss.mean()

class EarlyStopping:
    """Early stopping to prevent overfitting"""
    def __init__(self, patience=7, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.should_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        
        return self.should_stop


: 

In [ ]:
# ========================================== 
# SECTION 10: ENHANCED TRAINING LOOP
# ==========================================
class Trainer:
    def __init__(self, model, train_loader, val_loader, config):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        # Loss & optimizer
        class_counts = [sum(1 for _, l in train_loader.dataset if l == i) 
                       for i in range(config.NUM_CLASSES)]
        weights = torch.FloatTensor([1.0 / max(c, 1) for c in class_counts])
        weights = weights / weights.sum() * config.NUM_CLASSES
        self.criterion = FocalLoss(alpha=weights.to(self.device), gamma=2.0)
        
        self.optimizer = optim.AdamW(model.parameters(), 
                                     lr=config.LEARNING_RATE,
                                     weight_decay=config.WEIGHT_DECAY)
        
        self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=2, eta_min=1e-6
        )
        
        # Mixed precision
        self.scaler = GradScaler() if config.USE_MIXED_PRECISION else None
        
        # Early stopping
        self.early_stopping = EarlyStopping(patience=config.EARLY_STOPPING_PATIENCE)
        
        self.best_val_acc = 0.0
        self.history = defaultdict(list)
        
    def train_epoch(self, epoch):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{self.config.NUM_EPOCHS}')
        
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(self.device), labels.to(self.device)
            
            # Mixup augmentation
            if self.config.USE_ADVANCED_AUG and np.random.rand() < 0.5:
                images, labels_a, labels_b, lam = mixup_data(
                    images, labels, self.config.MIXUP_ALPHA
                )
                
                self.optimizer.zero_grad()
                
                if self.scaler:
                    with autocast():
                        outputs = self.model(images)
                        loss = mixup_criterion(self.criterion, outputs, 
                                             labels_a, labels_b, lam)
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 
                                                  self.config.GRADIENT_CLIP)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    outputs = self.model(images)
                    loss = mixup_criterion(self.criterion, outputs, 
                                         labels_a, labels_b, lam)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 
                                                  self.config.GRADIENT_CLIP)
                    self.optimizer.step()
            else:
                self.optimizer.zero_grad()
                
                if self.scaler:
                    with autocast():
                        outputs = self.model(images)
                        loss = self.criterion(outputs, labels)
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 
                                                  self.config.GRADIENT_CLIP)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 
                                                  self.config.GRADIENT_CLIP)
                    self.optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': f'{running_loss/(batch_idx+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
        
        epoch_loss = running_loss / len(self.train_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    @torch.no_grad()
    def validate_epoch(self, epoch):
        self.model.eval()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        all_probs = []
        
        pbar = tqdm(self.val_loader, desc='Validation')
        
        for images, labels in pbar:
            images, labels = images.to(self.device), labels.to(self.device)
            
            if self.scaler:
                with autocast():
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
            else:
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
            
            probs = F.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            running_loss += loss.item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
        
        epoch_loss = running_loss / len(self.val_loader)
        epoch_acc = 100. * accuracy_score(all_labels, all_preds)
        epoch_f1 = f1_score(all_labels, all_preds, average='weighted')
        
        return epoch_loss, epoch_acc, epoch_f1, all_preds, all_labels, all_probs
    
    def fit(self):
        print(f"\n🚀 Starting Training on {self.device}\n")
        
        for epoch in range(self.config.NUM_EPOCHS):
            # Train
            train_loss, train_acc = self.train_epoch(epoch)
            
            # Validate
            val_loss, val_acc, val_f1, val_preds, val_labels, val_probs = \
                self.validate_epoch(epoch)
            
            # Update scheduler
            self.scheduler.step()
            
            # Log metrics
            logger.log('train', epoch, {'loss': train_loss, 'acc': train_acc})
            logger.log('val', epoch, {
                'loss': val_loss, 
                'acc': val_acc, 
                'f1': val_f1
            })
            
            # Save history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            
            # Print summary
            print(f"\n Epoch {epoch+1} Summary:")
            print(f"   Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
            print(f"   Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%, F1: {val_f1:.4f}")
            print(f"   LR: {self.optimizer.param_groups[0]['lr']:.6f}\n")
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                checkpoint = {
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_acc': val_acc,
                    'val_f1': val_f1,
                    'config': self.config.__dict__
                }
                torch.save(checkpoint, 
                          os.path.join(Config.CHECKPOINT_DIR, 'best_model.pth'))
                print(f"💾 New best model saved! Val Acc: {val_acc:.2f}%\n")
            
            # Early stopping
            if self.early_stopping(val_loss):
                print(f" Early stopping triggered at epoch {epoch+1}")
                break
        
        print(f"\n Training Complete! Best Val Acc: {self.best_val_acc:.2f}%")
        return self.history

In [ ]:
# SECTION 11: INITIALIZE & TRAIN
# ==========================================
# Create datasets
train_dataset = AdvancedXRayDataset(
    Config.BASE_DIR, 'train', 
    transform=get_train_transforms(Config.IMG_SIZE)
)
val_dataset = AdvancedXRayDataset(
    Config.BASE_DIR, 'val',
    transform=get_val_transforms(Config.IMG_SIZE)
)
test_dataset = AdvancedXRayDataset(
    Config.BASE_DIR, 'test',
    transform=get_val_transforms(Config.IMG_SIZE)
)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, 
                         shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE,
                       shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE,
                        shuffle=False, num_workers=2, pin_memory=True)

print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

# Initialize model
model = AdvancedXRayModel(
    model_name=Config.MODEL_NAME,
    num_classes=Config.NUM_CLASSES,
    pretrained=Config.PRETRAINED
)

print(f" Model: {Config.MODEL_NAME}")
print(f" Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
trainer = Trainer(model, train_loader, val_loader, Config)
history = trainer.fit()


In [ ]:
# ========================================== 
# SECTION 12: COMPREHENSIVE EVALUATION
# ==========================================
def evaluate_model(model, test_loader, device):
    """Comprehensive model evaluation"""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Metrics
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print(f"\n Test Set Performance:")
    print(f"   Accuracy: {acc*100:.2f}%")
    print(f"   F1 Score: {f1:.4f}\n")
    
    # Classification report
    print(" Classification Report:")
    print(classification_report(all_labels, all_preds, 
                                target_names=Config.CLASSES, digits=4))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=Config.CLASSES, yticklabels=Config.CLASSES)
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(os.path.join(Config.LOG_DIR, 'confusion_matrix.png'), dpi=300)
    plt.show()
    
    # Per-class metrics
    print("\n Per-Class Accuracy:")
    class_acc = cm.diagonal() / cm.sum(axis=1)
    for i, cls in enumerate(Config.CLASSES):
        print(f"   {cls}: {class_acc[i]*100:.2f}%")
    
    # ROC curves
    all_probs = np.array(all_probs)
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    for i, cls in enumerate(Config.CLASSES):
        y_true = (np.array(all_labels) == i).astype(int)
        y_score = all_probs[:, i]
        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc = roc_auc_score(y_true, y_score)
        
        axes[i].plot(fpr, tpr, linewidth=2, label=f'AUC = {auc:.4f}')
        axes[i].plot([0, 1], [0, 1], 'k--', linewidth=1)
        axes[i].set_xlabel('False Positive Rate', fontsize=11)
        axes[i].set_ylabel('True Positive Rate', fontsize=11)
        axes[i].set_title(f'{cls} ROC Curve', fontsize=12, fontweight='bold')
        axes[i].legend(loc='lower right', fontsize=10)
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(Config.LOG_DIR, 'roc_curves.png'), dpi=300)
    plt.show()
    
    return acc, f1, all_preds, all_labels, all_probs

# Load best model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(os.path.join(Config.CHECKPOINT_DIR, 'best_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

# Evaluate
test_acc, test_f1, test_preds, test_labels, test_probs = evaluate_model(
    model, test_loader, device
)

In [ ]:
# ========================================== 
# SECTION 13: GRAD-CAM++ VISUALIZATION
# ==========================================
class GradCAMVisualizer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        target_layer = model.backbone.features[-1] \
            if hasattr(model.backbone, 'features') else model.backbone.layer4[-1]
        self.cam = GradCAMPlusPlus(model=model, target_layers=[target_layer])
    
    def visualize_batch(self, images, labels, predictions, n=4):
        fig, axes = plt.subplots(n, 3, figsize=(12, n*4))
        
        for i in range(min(n, len(images))):
            img_tensor = images[i].unsqueeze(0).to(self.device)
            
            # Original
            img = images[i].cpu().numpy().transpose(1, 2, 0)
            img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            img = np.clip(img, 0, 1)
            
            # GradCAM
            grayscale_cam = self.cam(input_tensor=img_tensor, targets=[predictions[i].item()])
            cam_image = show_cam_on_image(img, grayscale_cam[0], use_rgb=True)
            
            # Plot
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f'Original', fontsize=10)
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(cam_image)
            axes[i, 1].set_title(f'GradCAM++', fontsize=10)
            axes[i, 1].axis('off')
            
            axes[i, 2].imshow(grayscale_cam[0], cmap='jet')
            axes[i, 2].set_title(f'True: {Config.CLASSES[labels[i]]}\nPred: {Config.CLASSES[predictions[i]]}', 
                                fontsize=9)
            axes[i, 2].axis('off')
        
        plt.suptitle('Model Interpretability - GradCAM++ Heatmaps', 
                    fontsize=14, fontweight='bold', y=1.00)
        plt.tight_layout()
        plt.savefig(os.path.join(Config.LOG_DIR, 'gradcam_visualization.png'), 
                   dpi=300, bbox_inches='tight')
        plt.show()

# Visualize
visualizer = GradCAMVisualizer(model, device)
sample_images, sample_labels = next(iter(test_loader))
model.eval()
with torch.no_grad():
    outputs = model(sample_images.to(device))
    _, predictions = outputs.max(1)

visualizer.visualize_batch(sample_images[:4], sample_labels[:4], predictions[:4])

In [ ]:
# ========================================== 
# SECTION 14: DEPLOYMENT PACKAGE
# ==========================================
print("\n Creating deployment package...")

# Save model in multiple formats
torch.save(model.state_dict(), os.path.join(Config.CHECKPOINT_DIR, 'model_weights.pth'))
torch.save(model, os.path.join(Config.CHECKPOINT_DIR, 'model_complete.pth'))

# Model info
model_info = {
    'model_name': Config.MODEL_NAME,
    'accuracy': float(test_acc),
    'f1_score': float(test_f1),
    'classes': Config.CLASSES,
    'image_size': Config.IMG_SIZE,
    'parameters': sum(p.numel() for p in model.parameters()),
    'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'config': {k: v for k, v in Config.__dict__.items() if not k.startswith('_')}
}

with open(os.path.join(Config.CHECKPOINT_DIR, 'model_info.json'), 'w') as f:
    json.dump(model_info, f, indent=2)

# Create inference script
inference_code = '''
import torch
import torch.nn.functional as F
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

def load_model(model_path, device='cpu'):
    model = torch.load(model_path, map_location=device)
    model.eval()
    return model

def predict(model, image_path, device='cpu'):
    transform = A.Compose([
        A.Resize(256, 256),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = transform(image=image)['image'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        probs = F.softmax(output, dim=1)[0]
        pred_idx = probs.argmax().item()
        confidence = probs[pred_idx].item()
    
    classes = ['NORMAL', 'PNEUMONIA', 'COVID', 'TB']
    return classes[pred_idx], confidence, probs.cpu().numpy()

# Usage
model = load_model('model_complete.pth')
prediction, confidence, probabilities = predict(model, 'xray.jpg')
print(f"Prediction: {prediction} ({confidence*100:.2f}%)")
'''

with open(os.path.join(Config.CHECKPOINT_DIR, 'inference.py'), 'w') as f:
    f.write(inference_code)

# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(history['train_loss'], label='Train', marker='o', linewidth=2)
axes[0].plot(history['val_loss'], label='Val', marker='s', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training History - Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_acc'], label='Train', marker='o', linewidth=2)
axes[1].plot(history['val_acc'], label='Val', marker='s', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training History - Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['val_f1'], label='F1 Score', marker='d', 
            linewidth=2, color='green')
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('F1 Score', fontsize=12)
axes[2].set_title('Validation F1 Score', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(Config.LOG_DIR, 'training_history.png'), dpi=300)
plt.show()

# Summary
print("\n" + "="*70)
print("🎉 TRAINING COMPLETE - FINAL SUMMARY")
print("="*70)
print(f"Model: {Config.MODEL_NAME}")
print(f" Test Accuracy: {test_acc*100:.2f}%")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Total Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Training Time: {(time.time() - logger.start_time)/60:.1f} minutes")
print(f"\nSaved Files:")
print(f"   Model: {Config.CHECKPOINT_DIR}/model_complete.pth")
print(f"   Weights: {Config.CHECKPOINT_DIR}/model_weights.pth")
print(f"   Metadata: {Config.CHECKPOINT_DIR}/model_info.json")
print(f"   Inference Script: {Config.CHECKPOINT_DIR}/inference.py")
print(f"   Logs: {Config.LOG_DIR}/")
print("="*70)
print("Ready for deployment!")